Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [11]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [10]:
!pip install numerapi
!pip install vecstack;

In [9]:
pip install mljar-supervised

In [ ]:
##################################################################
##################### LIBRARIES ##################################
##################################################################

In [17]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool 
import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F

from vecstack import stacking

from sklearn import metrics




TypeError: ignored

In [ ]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [ ]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [ ]:
seed = 1
rand = np.random.seed(seed)

In [ ]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

2021-07-03 21:28:32,567 INFO numerapi.utils: starting download
./numerai_dataset_271.zip: 411MB [00:06, 64.9MB/s]                           
2021-07-03 21:28:38,910 INFO numerapi.base_api: unzipping file...


In [ ]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [ ]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 45.9 s, sys: 7.01 s, total: 53 s
Wall time: 54.7 s


In [ ]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [ ]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [ ]:
%%time
print('Call algorithm K-means')
kmeans = KMeans(n_clusters=10, random_state=rand_1, init = 'random')
print('Fitting algorithm K-means')
kmeans.fit(X)
print('Finished and Fitted')
X_clusters_kmeans = training_data
X_clusters_kmeans['k-means'] = kmeans.labels_

Call algorithm K-means
Fitting algorithm K-means
Finished and Fitted
CPU times: user 9min 15s, sys: 2min 41s, total: 11min 56s
Wall time: 8min 53s


In [ ]:
training_data["id_seq"] = list(range(len(training_data)))
tournament_data["id_seq"] = list(range(len(tournament_data)))

In [ ]:
tournament_data

,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,...,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target,id_seq
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.00,0.75,0.50,0.25,0.50,0.50,0.25,0.00,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.00,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,...,0.00,0.25,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.50,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00,0.25,0
n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.50,0.00,0.00,0.75,0.25,0.00,0.75,0.50,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.00,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,...,0.50,0.25,1.00,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.50,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50,0.50,1
n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.50,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,...,0.50,0.00,0.00,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00,1.00,2
n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.00,0.00,0.00,0.50,0.75,1.00,0.75,0.00,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.00,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.50,3
n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.00,0.50,0.50,0.25,0.25,0.50,0.50,1.00,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.00,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,...,0.50,0.50,0.00,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.00,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00,0.75,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffb091b15452291,eraX,live,0

In [ ]:
X_group_1 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 0]
X_group_2 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 1]
X_group_3 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 2]
X_group_4 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 3]
X_group_5 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 4]
X_group_6 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 5]
X_group_7 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 6]
X_group_8 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 7]
X_group_9 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 8]
X_group_10 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 9]

2021-07-03 21:39:00,549 INFO numexpr.utils: NumExpr defaulting to 4 threads.


In [ ]:
def Grouping_Models(df_train):
  
    df_train_all = df_train[feature_names]
    df_train_all[TARGET_NAME] = df_train['target'].values
    X = df_train_all[df_train_all.columns[0:-1]]
    Y = df_train_all[df_train_all.columns[-1]]
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.20,random_state=rand_3)
    model = LGBMRegressor(random_state=rand_1)
    model.fit(x_train, y_train, eval_set=[(x_test, y_test)])
    
    return model

In [ ]:
model_group_1 = Grouping_Models(X_group_1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0457931
[2]	valid_0's l2: 0.0457553
[3]	valid_0's l2: 0.0457702
[4]	valid_0's l2: 0.0457636
[5]	valid_0's l2: 0.0457753
[6]	valid_0's l2: 0.045778
[7]	valid_0's l2: 0.0457718
[8]	valid_0's l2: 0.0457882
[9]	valid_0's l2: 0.0458015
[10]	valid_0's l2: 0.0458102
[11]	valid_0's l2: 0.0458184
[12]	valid_0's l2: 0.0458357
[13]	valid_0's l2: 0.0458307
[14]	valid_0's l2: 0.0458431
[15]	valid_0's l2: 0.0458457
[16]	valid_0's l2: 0.0458414
[17]	valid_0's l2: 0.0458399
[18]	valid_0's l2: 0.0458465
[19]	valid_0's l2: 0.0458581
[20]	valid_0's l2: 0.0458679
[21]	valid_0's l2: 0.0458715
[22]	valid_0's l2: 0.0459001
[23]	valid_0's l2: 0.045899
[24]	valid_0's l2: 0.0459161
[25]	valid_0's l2: 0.0459308
[26]	valid_0's l2: 0.0459458
[27]	valid_0's l2: 0.0459529
[28]	valid_0's l2: 0.0459462
[29]	valid_0's l2: 0.0459539
[30]	valid_0's l2: 0.045939
[31]	valid_0's l2: 0.0459583
[32]	valid_0's l2: 0.0459751
[33]	valid_0's l2: 0.0459803
[34]	valid_0's l2: 0.0459789
[35]	valid_0's l2: 0.04597

In [ ]:
model_group_2 = Grouping_Models(X_group_2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0571647
[2]	valid_0's l2: 0.0571653
[3]	valid_0's l2: 0.0571644
[4]	valid_0's l2: 0.0571893
[5]	valid_0's l2: 0.0571811
[6]	valid_0's l2: 0.0571674
[7]	valid_0's l2: 0.0571775
[8]	valid_0's l2: 0.0571855
[9]	valid_0's l2: 0.0571926
[10]	valid_0's l2: 0.0571918
[11]	valid_0's l2: 0.0572039
[12]	valid_0's l2: 0.0572084
[13]	valid_0's l2: 0.0572
[14]	valid_0's l2: 0.0571814
[15]	valid_0's l2: 0.0572051
[16]	valid_0's l2: 0.0571978
[17]	valid_0's l2: 0.0572058
[18]	valid_0's l2: 0.0572067
[19]	valid_0's l2: 0.0572599
[20]	valid_0's l2: 0.0572658
[21]	valid_0's l2: 0.0572778
[22]	valid_0's l2: 0.0572791
[23]	valid_0's l2: 0.0572898
[24]	valid_0's l2: 0.0572929
[25]	valid_0's l2: 0.057286
[26]	valid_0's l2: 0.0572864
[27]	valid_0's l2: 0.0572892
[28]	valid_0's l2: 0.0572911
[29]	valid_0's l2: 0.0572788
[30]	valid_0's l2: 0.0572761
[31]	valid_0's l2: 0.0572801
[32]	valid_0's l2: 0.0573006
[33]	valid_0's l2: 0.0573028
[34]	valid_0's l2: 0.057316
[35]	valid_0's l2: 0.05732
[

In [ ]:
model_group_2 = Grouping_Models(X_group_2)
model_group_3 = Grouping_Models(X_group_3)
model_group_4 = Grouping_Models(X_group_4)
model_group_5 = Grouping_Models(X_group_5)
model_group_6 = Grouping_Models(X_group_6)
model_group_7 = Grouping_Models(X_group_7)
model_group_8 = Grouping_Models(X_group_8)
model_group_9 = Grouping_Models(X_group_9)
model_group_10 = Grouping_Models(X_group_10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.05605
[2]	valid_0's l2: 0.0560649
[3]	valid_0's l2: 0.0560677
[4]	valid_0's l2: 0.0560611
[5]	valid_0's l2: 0.05608
[6]	valid_0's l2: 0.0560726
[7]	valid_0's l2: 0.0560751
[8]	valid_0's l2: 0.0560805
[9]	valid_0's l2: 0.0560835
[10]	valid_0's l2: 0.0560796
[11]	valid_0's l2: 0.0560702
[12]	valid_0's l2: 0.0560677
[13]	valid_0's l2: 0.0560605
[14]	valid_0's l2: 0.056078
[15]	valid_0's l2: 0.0560811
[16]	valid_0's l2: 0.0560482
[17]	valid_0's l2: 0.0560411
[18]	valid_0's l2: 0.0560351
[19]	valid_0's l2: 0.0560488
[20]	valid_0's l2: 0.056045
[21]	valid_0's l2: 0.0560536
[22]	valid_0's l2: 0.0560687
[23]	valid_0's l2: 0.0560748
[24]	valid_0's l2: 0.0560719
[25]	valid_0's l2: 0.0560834
[26]	valid_0's l2: 0.0560953
[27]	valid_0's l2: 0.0560821
[28]	valid_0's l2: 0.0560659
[29]	valid_0's l2: 0.0560935
[30]	valid_0's l2: 0.0560887
[31]	valid_0's l2: 0.0560861
[32]	valid_0's l2: 0.0560871
[33]	valid_0's l2: 0.0561011
[34]	valid_0's l2: 0.0561187
[35]	valid_0's l2: 0.056107
[

In [ ]:
########################## PREDICTIONS ON LEBELS ##############################

In [ ]:
labels_tournament = kmeans.predict(tournament_data[feature_names])
tournament_data['k-means'] = labels_tournament

In [ ]:
X_group_1_td = tournament_data.loc[tournament_data['k-means'] == 0]
X_group_2_td = tournament_data.loc[tournament_data['k-means'] == 1]
X_group_3_td = tournament_data.loc[tournament_data['k-means'] == 2]
X_group_4_td = tournament_data.loc[tournament_data['k-means'] == 3]
X_group_5_td = tournament_data.loc[tournament_data['k-means'] == 4]
X_group_6_td = tournament_data.loc[tournament_data['k-means'] == 5]
X_group_7_td = tournament_data.loc[tournament_data['k-means'] == 6]
X_group_8_td = tournament_data.loc[tournament_data['k-means'] == 7]
X_group_9_td = tournament_data.loc[tournament_data['k-means'] == 8]
X_group_10_td = tournament_data.loc[tournament_data['k-means'] == 9]

In [ ]:
prediction_group_1 = pd.DataFrame()
prediction_group_1[PREDICTION_NAME] = model_group_1.predict(X_group_1_td[feature_names])
prediction_group_1["id_seq"] = X_group_1_td["id_seq"].values

In [ ]:
prediction_group_2 = pd.DataFrame()
prediction_group_2[PREDICTION_NAME] = model_group_2.predict(X_group_2_td[feature_names])
prediction_group_2["id_seq"] = X_group_2_td["id_seq"].values

In [ ]:
prediction_group_3 = pd.DataFrame()
prediction_group_3[PREDICTION_NAME] = model_group_3.predict(X_group_3_td[feature_names])
prediction_group_3["id_seq"] = X_group_3_td["id_seq"].values

In [ ]:
prediction_group_4 = pd.DataFrame()
prediction_group_4[PREDICTION_NAME] = model_group_4.predict(X_group_4_td[feature_names])
prediction_group_4["id_seq"] = X_group_4_td["id_seq"].values

In [ ]:
prediction_group_5 = pd.DataFrame()
prediction_group_5[PREDICTION_NAME] = model_group_5.predict(X_group_5_td[feature_names])
prediction_group_5["id_seq"] = X_group_5_td["id_seq"].values

In [ ]:
prediction_group_6 = pd.DataFrame()
prediction_group_6[PREDICTION_NAME] = model_group_6.predict(X_group_6_td[feature_names])
prediction_group_6["id_seq"] = X_group_6_td["id_seq"].values

In [ ]:
prediction_group_7 = pd.DataFrame()
prediction_group_7[PREDICTION_NAME] = model_group_7.predict(X_group_7_td[feature_names])
prediction_group_7["id_seq"] = X_group_7_td["id_seq"].values

In [ ]:
prediction_group_8 = pd.DataFrame()
prediction_group_8[PREDICTION_NAME] = model_group_8.predict(X_group_8_td[feature_names])
prediction_group_8["id_seq"] = X_group_8_td["id_seq"].values

In [ ]:
prediction_group_9 = pd.DataFrame()
prediction_group_9[PREDICTION_NAME] = model_group_9.predict(X_group_9_td[feature_names])
prediction_group_9["id_seq"] = X_group_9_td["id_seq"].values

In [ ]:
prediction_group_10 = pd.DataFrame()
prediction_group_10[PREDICTION_NAME] = model_group_10.predict(X_group_10_td[feature_names])
prediction_group_10["id_seq"] = X_group_10_td["id_seq"].values

In [ ]:
########################## UNION OF PREDICTIONS ##############################

In [ ]:
vertical_stack = pd.concat([prediction_group_1, prediction_group_2,prediction_group_3,prediction_group_4,prediction_group_5
                            ,prediction_group_6,prediction_group_7,prediction_group_8,prediction_group_9
                            ,prediction_group_10], axis=0)
vertical_stack = vertical_stack.sort_values("id_seq")

In [ ]:
tournament_data[PREDICTION_NAME] = vertical_stack[PREDICTION_NAME].values

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.497072
n000920ed083903f    0.504768
n0038e640522c4a6    0.535283
n004ac94a87dc54b    0.484379
n0052fe97ea0c05f    0.495851
                      ...   
nffb091b15452291    0.493713
nffc0338378383a7    0.502288
nffcfc256cb1ec1a    0.459003
nffdb7467dc351b9    0.543859
nffec2680b381924    0.503026
Name: prediction, Length: 1757987, dtype: float64

In [ ]:
########################## UNION OF PREDICTIONS ##############################

In [ ]:
import numerapi
import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F

In [ ]:
def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)

def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer

def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.497072
n000920ed083903f    0.504768
n0038e640522c4a6    0.535283
n004ac94a87dc54b    0.484379
n0052fe97ea0c05f    0.495851
                      ...   
nffb091b15452291    0.493713
nffc0338378383a7    0.502288
nffcfc256cb1ec1a    0.459003
nffdb7467dc351b9    0.543859
nffec2680b381924    0.503026
Name: prediction, Length: 1757987, dtype: float64

In [ ]:
data_rfe_10 = reduce_all_exposures(tournament_data,
                                   [PREDICTION_NAME],
                                   neutralizers=feature_names,
                                   era_col="era",
                                   max_exp=0.15)

era121 
era122 
era123 
era124 
era125 
era126 
era127 
era128 
era129 
era130 
era131 
era132 
era575 
era576 
era577 
era578 
era579 
era580 
era581 
era582 
era583 
era584 
era585 
era586 
era587 
era588 
era589 
era590 
era591 
era592 
era593 
era594 
era595 
era596 
era597 
era598 
era599 
era600 
era601 
era602 
era603 
era604 
era605 
era606 
era607 
era608 
era609 
era610 
era611 
era612 
era613 
era614 
era615 
era616 
era617 
era618 
era619 
era620 
era621 
era622 
era623 
era624 
era625 
era626 
era627 
era628 
era629 
era630 
era631 
era632 
era633 
era634 
era635 
era636 
era637 
era638 
era639 
era640 
era641 
era642 
era643 
era644 
era645 
era646 
era647 
era648 
era649 
era650 
era651 
era652 
era653 
era654 
era655 
era656 
era657 
era658 
era659 
era660 
era661 
era662 
era663 
era664 
era665 
era666 
era667 
era668 
era669 
era670 
era671 
era672 
era673 
era674 
era675 
era676 
era677 
era678 
era679 
era680 
era681 
era682 
era683 
era684 
era685 
era686 
era687 


In [ ]:
# replace prediction with reduced feature exposure prediction and rescale to [0,1]
tournament_data[PREDICTION_NAME] = data_rfe_10[PREDICTION_NAME]
tournament_data[PREDICTION_NAME] -= tournament_data[PREDICTION_NAME].min()
tournament_data[PREDICTION_NAME] /= tournament_data[PREDICTION_NAME].max()

In [ ]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [ ]:
tournament_data[PREDICTION_NAME].to_csv("FisaGol_4.csv")

In [ ]:
#save your prediction file locally

In [ ]:
!cp FisaGol_4.csv "drive/My Drive/PHOENIXSIGMA/"